In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.listeners

In [ ]:
# export

import time
import os
import signal
import threading
from threading import Thread
from pymemri.plugin.states import RUN_COMPLETED, RUN_FAILED

In [ ]:
# export

class StatusListener:
    def __init__(self, client, run_id, status, callback, interval=5, verbose=False):
        self.client = client
        self.run_id = run_id
        self.status = status
        self.callback = callback
        self.interval = interval
        self.verbose = verbose
        self.running = True
        
    def stop(self):
        if self.verbose:
            print("Stopping listener...", flush=True)
        self.running = False
        
    def run(self):
        if self.verbose:
            print(f"Listening for status='{self.status}' on Item {self.run_id}", flush=True)
        
        while self.running and threading.main_thread().is_alive():
            time.sleep(self.interval)
            try:
                run = self.client.get(self.run_id)
                if self.verbose:
                    print("run status:", run.status, flush=True)
                if run.status == self.status:
                    self.callback()
            except Exception as e:
                print(f"Could not get run in status listener: {e}")

In [ ]:
# export
def force_exit_callback():
    print("Status aborted, killing plugin...", flush=True)
    pid = os.getpid()
    os.kill(pid, signal.SIGINT)

def get_abort_plugin_listener(client, run_id, **kwargs):
    listener = StatusListener(
        client=client,
        run_id=run_id,
        status="aborted",
        callback=force_exit_callback,
        **kwargs
    )
    thread = Thread(
        target=listener.run,
    )
    thread.start()

    return listener

In [ ]:
# hide
# skip
from pymemri.pod.client import PodClient
from pymemri.data.schema import PluginRun

def test_callback():
    """Print Callback and kill listener."""
    print("Callback")
    os.kill(os.getpid(), signal.SIGINT)

client = PodClient()
run = PluginRun(containerImage="", status="started")
client.create(run)

listener = get_abort_plugin_listener(client, run.id, verbose=True)

for i in range(10):
    time.sleep(1)
    if i == 5:
        run = client.get(run.id)
        run.status = "aborted"
        client.update_item(run)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
